# Bond graph features model - Impairment - Exploration, calibration, test

This notebook uses as inputs the outputs from the pipeline ('08_preproc_pipeline_2.ipynb' notebook) and performs model calibration and general exploration for the transaction credit events prediction.

In [1]:
import pandas as pd
import numpy as np
import pickle
from os import environ

from sklearn.model_selection import cross_val_predict, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from scripts_viz.visualization_utils import *
from scripts_ml.models_utils import *

from bokeh.io import show, output_notebook
output_notebook()

import datetime as dt

Loading BokehJS ...

### TRAIN Data import

In [2]:
#importing data
user=environ["USERNAME"]
prefix = 'shuffle_imp_bg_'
trainfile = '_traindata'
testfile = '_testdata'
postfix = '_190721_1655'

#from home
datafolder = ".."+"/data/preproc_traintest/"

[X_train, y_train, feature_labels] = pd.read_pickle(datafolder+prefix+trainfile+postfix+'.pkl') 
[X_test, y_test, feature_labels] = pd.read_pickle(datafolder+prefix+testfile+postfix+'.pkl') 

In [3]:
X_train_df = pd.DataFrame(X_train, columns=feature_labels)
X_train_df.head()

,has_purchase,dd_value_date,cd_lent_c,cd_repaid_c,cd_impaired1_c,cd_pastdue90_c,cd_trend_a,c_lent_c,c_repaid_c,c_impaired1_c,...,flow_shock_p180,p180_d_node_flow,p180_energy,invoice_amount,purchase_amount,currency_Britisches Pfund,currency_Euro,currency_Schweizer Franken,currency_US-Dollar,invoice_date
0,0.478098,-0.145273,-0.528606,-0.493910,-0.17567,-0.214037,-1.385604,-1.009960,-0.893308,-0.554907,...,-0.376595,-0.301636,-0.221484,0.967374,0.678658,0.0,0.0,1.0,0.0,-1.870522
1,0.478098,-0.145273,-0.302102,-0.493910,-0.17567,-0.214037,-0.960531,-0.938481,-0.893308,-0.554907,...,0.870160,-0.301636,-0.261344,2.014475,0.905137,0.0,1.0,0.0,0.0,-0.606708
2,0.478098,-0.145273,-0.457419,-0.368922,-0.17567,-0.214037,0.997265,-0.922398,-0.788296,-0.554907,...,-0.375489,-0.301636,-0.221484,-0.012316,0.466758,0.0,0.0,1.0,0.0,0.315048
3,0.478098,-0.145273,-0.502720,-0.460580,-0.17567,-0.214037,1.624837,-0.989410,-0.861077,-0.554907,...,-0.371151,-0.301636,-0.221484,-0.829188,0.290076,0.0,0.0,1.0,0.0,0.675109
4,0.478098,-0.145273,-0.522134,-0.443915,-0.17567,-0.214037,1.315468,0.274891,0.239989,0.079527,...,-0.196853,-0.301636,-0.221484,1.362819,0.764189,0.0,0.0,1.0,0.0,0.523883


In [4]:
X_train_df.columns

Index(['has_purchase', 'dd_value_date', 'cd_lent_c', 'cd_repaid_c',
       'cd_impaired1_c', 'cd_pastdue90_c', 'cd_trend_a', 'c_lent_c',
       'c_repaid_c', 'c_impaired1_c', 'c_pastdue90_c', 'c_trend_a',
       'd_repaid_c', 'd_impaired1_c', 'd_pastdue90_c', 'd_trend_a',
       'd_we_payment_share', 'flow_shock_imp1', 'imp_c_node_eff', 'imp_energy',
       'imp_d_node_flow', 'flow_shock_p90', 'p90_c_node_eff', 'p90_energy',
       'p90_d_node_flow', 'flow_shock_p180', 'p180_d_node_flow', 'p180_energy',
       'invoice_amount', 'purchase_amount', 'currency_Britisches Pfund',
       'currency_Euro', 'currency_Schweizer Franken', 'currency_US-Dollar',
       'invoice_date'],
      dtype='object')

In [5]:
X_train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
has_purchase,46095.0,-2.067704e-08,1.000011,-2.091622,0.478098,0.478098,0.478098,0.478098
dd_value_date,46095.0,3.199733e-09,1.000011,-0.145273,-0.145273,-0.145273,-0.145273,85.601524
cd_lent_c,46095.0,1.108227e-09,1.000011,-0.567435,-0.528606,-0.431533,-0.017355,4.393315
cd_repaid_c,46095.0,-1.870823e-08,1.000011,-0.493910,-0.493910,-0.418917,-0.093949,4.377090
cd_impaired1_c,46095.0,-1.428856e-10,1.000011,-0.175670,-0.175670,-0.175670,-0.175670,8.903720
cd_pastdue90_c,46095.0,-4.453377e-09,1.000011,-0.214037,-0.214037,-0.214037,-0.214037,6.398803
cd_trend_a,46095.0,-1.419782e-08,1.000011,-3.920882,-0.678948,-0.036710,0.712322,3.604046
c_lent_c,46095.0,-2.698500e-08,1.000011,-1.037659,-0.801775,-0.315712,0.566171,3.869435
c_repaid_c,46095.0,1.823345e-08,1.000011,-0.893308,-0.788296,-0.336017,0.435457,4.005999
c_impaired1_c,46095.0,1.572906e-08,1.000011,-0.554907,-0.554907,-0.343429,0.079527,4.806320


### SGD Classifier

In [6]:
#Linear model Stochastic Gradient Descent
sgd_clf = SGDClassifier(random_state=42, max_iter=250, loss='log', tol=0.0001) 
sgd_clf.fit(X_train, y_train)
sgd_clf_diag = model_diag(sgd_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.811
Confusion matrix: 
[[0.99849 0.00151]
 [0.01861 0.00284]]


In [23]:
y_score_sgd = sgd_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd)
print(cm)
sgd_test_diag = model_oostest(sgd_clf,X_test, y_test)
sgd_test_auc = plot_rocs([sgd_clf_diag, sgd_test_diag], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc)

[[11265    30]
 [  195    32]]
[[11265    30]
 [  195    32]]
AUC 0.819


In [9]:
#save_sk_model(sgd_clf, "../data/models/", 'sgd', 'bondgraph')

Saving model to ../data/models/bondgraph_sgd_190710_147.pkl


In [7]:
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

# max Number of iterations
max_iter = [100, 180, 200, 250, 300, 320, 350] 
# learning rate
learning_rate = ['constant', 'optimal', 'adaptive']
#eta
eta0 = [0.00001, 0.0001, 0.001, 0.01]
#loss
loss = ['log', 'hinge', 'perceptron']

sgd_random_grid = {'max_iter': max_iter,
               'learning_rate': learning_rate,
                  'eta0': eta0,
                  'loss':loss}

sgd = SGDClassifier(tol=0.0001)

sgd_rsearch = RandomizedSearchCV(estimator = sgd, param_distributions = sgd_random_grid, n_iter = 200, cv = 3, 
                                 verbose=2, random_state=42, n_jobs =7, scoring=scoring, refit='AUC')
# Fit the random search model
sgd_rsearch.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    3.7s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:   10.8s
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed:   23.5s
[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:   41.1s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=7,
          param_distributions={'max_iter': [100, 180, 200, 250, 300, 320, 350], 'learning_rate': ['constant', 'optimal', 'adaptive'], 'eta0': [1e-05, 0.0001, 0.001, 0.01], 'loss': ['log', 'hinge', 'perceptron']},
          pre_dispatch='2*n_jobs', random_state=42, refit='AUC',
          return_train_score='warn',
          scoring={'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)},
          verbose=2)

In [8]:
sgd_rsearch.best_params_

{'max_iter': 320, 'loss': 'log', 'learning_rate': 'optimal', 'eta0': 1e-05}

In [9]:
#Linear model Stochastic Gradient Descent
sgd_rs = SGDClassifier(random_state=42, max_iter=350, loss='log', learning_rate='optimal', eta0=1e-05, tol=0.0001) 
sgd_rs.fit(X_train, y_train)
sgd_rs_diag = model_diag(sgd_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.811
Confusion matrix: 
[[0.99849 0.00151]
 [0.01861 0.00284]]


In [10]:
y_score_sgd_rs = sgd_rs.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd_rs)
print(cm)
sgd_test_diag_rs = model_oostest(sgd_rs,X_test, y_test)
sgd_test_auc_rs = plot_rocs([sgd_rs_diag, sgd_test_diag_rs], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc_rs)

[[11265    30]
 [  195    32]]
[[11265    30]
 [  195    32]]
AUC 0.819


### Random Forest Classifier

In [11]:
#random forest - benchmark
n_estimators = 200
max_leaf_nodes = 40
rf_clf = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                               max_leaf_nodes=max_leaf_nodes,
                               class_weight="balanced",
                               n_jobs=7)

rf_clf.fit(X_train, y_train)
rf_clf_diag = model_diag(rf_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.969
Confusion matrix: 
[[0.93538 0.06462]
 [0.0025  0.01895]]


In [12]:
y_score_rf = rf_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_rf)
print(cm)
rf_test_diag = model_oostest(rf_clf,X_test, y_test)
rf_test_auc = plot_rocs([rf_clf_diag, rf_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_test_auc)

[[10552   743]
 [   29   198]]
[[10552   743]
 [   29   198]]
AUC 0.967


In [32]:
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

# Number of trees in random forest
n_estimators = [150, 180, 200, 250, 280, 300, 350] 
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 10, 20]
# Maximum number of levels in tree
max_depth = [100,200, None]
#Max leaf nodes
max_leaf_nodes = [10,20,40,60]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
              'max_leaf_nodes':max_leaf_nodes}
print(random_grid)

{'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]}


In [14]:
rf = RandomForestClassifier() #base model
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 80, cv = 3, 
                               verbose=2, random_state=42, n_jobs =7, scoring=scoring, refit='AUC')
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  1.0min
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:  5.8min
[Parallel(n_jobs=7)]: Done 240 out of 240 | elapsed:  9.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=80, n_jobs=7,
          param_distributions={'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]},
          pre_dispatch='2*n_jobs', random_state=42, refit='AUC',
          return_train_score='warn',
          scoring={'AUC': 'roc_auc', 'Accurac

In [33]:
rf_random.best_params_

{'n_estimators': 280,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 200,
 'bootstrap': True}

In [34]:
#first randomized search
rs1 = {'n_estimators': 280,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_leaf_nodes':60,
 'max_depth': 100,
 'bootstrap': True}

#second randomized search 
rs2 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#third randomized search 
rs3 = {'n_estimators': 280,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 200,
 'bootstrap': True}

In [17]:
#random_forest from rand
n_estimators=rs3['n_estimators']
min_samples_split=rs3['min_samples_split']
min_samples_leaf=rs3['min_samples_leaf']
max_leaf_nodes=rs3['max_leaf_nodes']
max_features=rs3['max_features']
max_depth=rs3['max_depth']
bootstrap=rs3['bootstrap']




rf_rs = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                                max_features=max_features,
                                min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf,
                               class_weight="balanced", bootstrap=bootstrap,
                               n_jobs=7)
rf_rs.fit(X_train, y_train)
rf_rs_diag = model_diag(rf_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.971
Confusion matrix: 
[[0.99422 0.00578]
 [0.00791 0.01354]]


In [19]:
y_score_rf_rs = rf_rs.predict(X_test)
cm = confusion_matrix(y_test, y_score_rf_rs)
rf_rs_test_diag = model_oostest(rf_rs,X_test, y_test)
rf_rs_test_auc = plot_rocs([rf_rs_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_rs_test_auc)

[[11224    71]
 [   98   129]]
AUC 0.973


In [20]:
#save_sk_model(rf_rs, "../data/models/", 'rf_imp', 'enriched')

Saving model to ../data/models/enriched_rf_imp_190710_1535.pkl


### Training Performance Comparison

In [21]:
rf_sgd_roc = plot_rocs([sgd_clf_diag, rf_rs_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)

### Test Performance Comparison

In [24]:
rf_sgd_roc = plot_rocs([sgd_test_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)

## REPEATING THE EXPERIMENT IN TIME MODE

In [35]:
prefix = 'time_2018-04-30_imp_bgt_'
postfix = '_190710_1539'
[X_train, y_train, feature_labels] = pd.read_pickle(datafolder+prefix+trainfile+postfix+'.pkl') 
[X_test, y_test, feature_labels] = pd.read_pickle(datafolder+prefix+testfile+postfix+'.pkl') 

### SGD Classifier

In [36]:
#Linear model Stochastic Gradient Descent
sgd_clf = SGDClassifier(random_state=42, max_iter=250, loss='log', tol=0.0001) 
sgd_clf.fit(X_train, y_train)
sgd_clf_diag = model_diag(sgd_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.754
Confusion matrix: 
[[0.99048 0.00952]
 [0.01278 0.00286]]


In [37]:
y_score_sgd = sgd_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd)
print(cm)
sgd_test_diag = model_oostest(sgd_clf,X_test, y_test)
sgd_test_auc = plot_rocs([sgd_clf_diag, sgd_test_diag], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc)

[[10995    38]
 [  419    66]]
[[10995    38]
 [  419    66]]
AUC 0.785


In [38]:
#save_sk_model(sgd_clf, "../data/models/", 'sgd', 'benchmark')

In [39]:
scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

# max Number of iterations
max_iter = [100, 180, 200, 250, 300, 320, 350] 
# learning rate
learning_rate = ['constant', 'optimal', 'adaptive']
#eta
eta0 = [0.00001, 0.0001, 0.001, 0.01]
#loss
loss = ['log', 'hinge', 'perceptron']

sgd_random_grid = {'max_iter': max_iter,
               'learning_rate': learning_rate,
                  'eta0': eta0,
                  'loss':loss}

sgd = SGDClassifier(tol=0.0001)

sgd_rsearch = RandomizedSearchCV(estimator = sgd, param_distributions = sgd_random_grid, n_iter = 200, cv = 3, verbose=2, 
                                 random_state=42, n_jobs =7, scoring=scoring, refit='AUC')
# Fit the random search model
sgd_rsearch.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    3.6s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:   15.1s
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed:   37.4s
[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:  1.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=7,
          param_distributions={'max_iter': [100, 180, 200, 250, 300, 320, 350], 'learning_rate': ['constant', 'optimal', 'adaptive'], 'eta0': [1e-05, 0.0001, 0.001, 0.01], 'loss': ['log', 'hinge', 'perceptron']},
          pre_dispatch='2*n_jobs', random_state=42, refit='AUC',
          return_train_score='warn',
          scoring={'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)},
          verbose=2)

In [40]:
sgd_rsearch.best_params_

{'max_iter': 250, 'loss': 'log', 'learning_rate': 'adaptive', 'eta0': 0.01}

In [41]:
#Linear model Stochastic Gradient Descent
sgd_rs = SGDClassifier(random_state=42, max_iter=250, loss='log', learning_rate='adaptive', eta0=0.01, tol=0.0001) 
sgd_rs.fit(X_train, y_train)
sgd_rs_diag = model_diag(sgd_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.782
Confusion matrix: 
[[0.99537 0.00463]
 [0.01304 0.0026 ]]


In [42]:
sgd_test_diag_rs = model_oostest(sgd_rs,X_test, y_test)
sgd_test_auc_rs = plot_rocs([sgd_rs_diag, sgd_test_diag_rs], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc_rs)

[[10996    37]
 [  411    74]]
AUC 0.807


### Random Forest Classifier

In [43]:
#random forest - benchmark
n_estimators = 200
max_leaf_nodes = 40
rf_clf = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                               max_leaf_nodes=max_leaf_nodes,
                               class_weight="balanced",
                               n_jobs=7)

rf_clf.fit(X_train, y_train)
rf_clf_diag = model_diag(rf_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.926
Confusion matrix: 
[[0.91787 0.08213]
 [0.00386 0.01179]]


In [44]:
rf_test_diag = model_oostest(rf_clf,X_test, y_test)
rf_test_auc = plot_rocs([rf_clf_diag, rf_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_test_auc)

[[10113   920]
 [   84   401]]
AUC 0.956


In [45]:
# Number of trees in random forest
n_estimators = [150, 180, 200, 250, 280, 300, 350] 
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 10, 20]
# Maximum number of levels in tree
max_depth = [100,200, None]
#Max leaf nodes
max_leaf_nodes = [10,20,40,60]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
              'max_leaf_nodes':max_leaf_nodes}
print(random_grid)

{'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]}


In [47]:
rf = RandomForestClassifier() #base model
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs =7, scoring=scoring, refit='AUC')
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  2.0min
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed: 10.5min
[Parallel(n_jobs=7)]: Done 300 out of 300 | elapsed: 20.6min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=7,
          param_distributions={'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]},
          pre_dispatch='2*n_jobs', random_state=42, refit='AUC',
          return_train_score='warn',
          scoring={'AUC': 'roc_auc', 'Accura

In [48]:
rf_random.best_params_

{'n_estimators': 280,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 10,
 'max_features': 10,
 'max_depth': 100,
 'bootstrap': False}

In [49]:
#first randomized search_imp
rs1 = {'n_estimators': 280,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_leaf_nodes':60,
 'max_depth': 100,
 'bootstrap': True}

#second randomized search_imp
rs2 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#third randomized search_p90
rs3 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#fourth randomized search_p90_time
rs4 = {'n_estimators': 300,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 10,
 'max_features': 'auto',
 'max_depth': 200,
 'bootstrap': True}

#fifth randomized search_imp_time
rs5 = {'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 10,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

#sixth randomized search_imp_time bgt
rs6 = {'n_estimators': 280,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 10,
 'max_features': 10,
 'max_depth': 100,
 'bootstrap': False}

In [50]:
#random_forest from rand
n_estimators=rs6['n_estimators']
min_samples_split=rs6['min_samples_split']
min_samples_leaf=rs6['min_samples_leaf']
max_leaf_nodes=rs6['max_leaf_nodes']
max_features=rs6['max_features']
max_depth=rs6['max_depth']
bootstrap=rs6['bootstrap']




rf_rs = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                                max_features=max_features,
                                min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf,
                               class_weight="balanced", bootstrap=bootstrap,
                               n_jobs=7)
rf_rs.fit(X_train, y_train)
rf_rs_diag = model_diag(rf_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.810
Confusion matrix: 
[[0.99183 0.00817]
 [0.01304 0.0026 ]]


In [52]:
y_score_rf_rs = rf_rs.predict(X_test)
rf_rs_test_diag = model_oostest(rf_rs,X_test, y_test)
rf_rs_test_auc = plot_rocs([rf_rs_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_rs_test_auc)

[[11015    18]
 [  394    91]]
AUC 0.953


In [53]:
#save_sk_model(rf_rs, "../data/models/", 'rf_imp_time', 'enriched')

Saving model to ../data/models/enriched_rf_imp_time_190710_1619.pkl


### Training Performance Comparison

In [54]:
rf_sgd_roc = plot_rocs([sgd_clf_diag, rf_rs_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)

### Test Performance Comparison

In [55]:
rf_sgd_roc = plot_rocs([sgd_test_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)